<a href="https://colab.research.google.com/github/jdospina/jump-diffusion-estimation/blob/main/notebooks/sp500_jump_diffusion_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📈 Análisis del S&P 500 con un Modelo de Difusión con Saltos

Este notebook es un tutorial completo sobre cómo aplicar un **modelo de difusión con saltos (jump-diffusion)** para analizar los rendimientos del índice S&P 500. Los mercados financieros a menudo exhiben movimientos bruscos (saltos) que no pueden ser capturados por modelos de difusión simples como el de Black-Scholes. Nuestro modelo busca capturar tanto la volatilidad continua como estos saltos abruptos.

El flujo de trabajo será el siguiente:
1.  **Obtención y Preparación de Datos**: Descargaremos los datos históricos del S&P 500 (`^GSPC`) y calcularemos los rendimientos logarítmicos diarios.
2.  **Estimación de Parámetros**: Ajustaremos nuestro modelo de difusión con saltos a los datos históricos para estimar sus parámetros clave (`μ`, `σ`, probabilidad de salto, etc.).
3.  **Análisis de Resultados**: Interpretaremos los parámetros estimados y evaluaremos la calidad del ajuste del modelo.
4.  **Simulación y Comparación**: Usaremos los parámetros estimados para simular una nueva serie de rendimientos y compararemos su distribución con la de los datos reales para validar nuestro modelo.

## 1. Preparación del Entorno y Carga de Datos

In [ ]:
# Instalación de la librería jump-diffusion-estimation desde GitHub (última versión)
%pip install git+https://github.com/jdospina/jump-diffusion-estimation.git

In [ ]:
%pip install yfinance matplotlib seaborn -q

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from jump_diffusion.estimation import JumpDiffusionEstimator
from jump_diffusion.simulation import JumpDiffusionSimulator

# Configuración de estilo para los gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

### Descarga de Datos del S&P 500
Utilizaremos la librería `yfinance` para descargar los precios de cierre ajustados del índice S&P 500 (`^GSPC`) desde 2015 hasta principios de 2024.

In [ ]:
data = yf.download('^GSPC', start='2015-01-01', end='2024-01-01', progress=False)
prices = data['Close']['^GSPC'].dropna()

print("Primeros 5 precios del S&P 500:")
print(prices.head())

Veamos la serie original:

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(prices.index, prices.values, alpha=0.8, color='b')
plt.title('Serie del valor de cierre del S&P 500')
plt.xlabel('Fecha')
plt.ylabel('Valor')
plt.grid(True)
plt.show()

### Cálculo de Rendimientos Logarítmicos
Los modelos financieros suelen trabajar con los **rendimientos logarítmicos** (`log(P_t) - log(P_{t-1})`), ya que tienen propiedades estadísticas más deseables (como la estacionariedad). Estos rendimientos representarán los "incrementos" en nuestro modelo.

In [ ]:
log_returns = np.diff(np.log(prices.values))

# El intervalo de tiempo (dt) es diario. Asumimos 252 días de trading al año.
dt = 1/252

print(f"Se han calculado {len(log_returns)} rendimientos logarítmicos diarios.")

### Visualización de los Rendimientos
Un primer vistazo a la serie de rendimientos nos permite observar la volatilidad y los picos extremos (posibles saltos), como los ocurridos durante la crisis de COVID-19 en 2020.

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(prices.index[1:], log_returns, alpha=0.8, color='b')
plt.title('Rendimientos Logarítmicos Diarios del S&P 500')
plt.xlabel('Fecha')
plt.ylabel('Log Return')
plt.grid(True)
plt.show()

Distribución de los incrementos

In [ ]:
plt.figure(figsize=(14, 7))
plt.hist(log_returns, color='b', bins = 100)
plt.title('Rendimientos Logarítmicos Diarios del S&P 500')
plt.xlabel('Incrementos')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()

## 2. Estimación del Modelo de Difusión con Saltos

In [ ]:
# Inicializamos el estimador con los rendimientos y el dt
estimator = JumpDiffusionEstimator(log_returns, dt)

# Realizamos la estimación por máxima verosimilitud
results = estimator.estimate()

# Mostramos un reporte de diagnóstico con los resultados
estimator.diagnostics()

## 3. Simulación y Comparación de Distribuciones

Ahora, el paso crucial: ¿qué tan bien captura nuestro modelo la realidad? Para responder a esto, simularemos un proceso de difusión con saltos utilizando los parámetros que acabamos de estimar. Luego, compararemos la distribución de los rendimientos simulados con la distribución de los rendimientos reales del S&P 500.

In [ ]:
# Extraemos los parámetros estimados
estimated_params = results['parameters']

# Creamos un simulador con estos parámetros
simulator = JumpDiffusionSimulator(**estimated_params)

# Simulamos una trayectoria con la misma cantidad de pasos que los datos originales
T = len(log_returns) * dt
n_steps = len(log_returns)

_, simulated_path, _ = simulator.simulate_path(T=T, n_steps=n_steps, x0=prices.iloc[0])

# Calculamos los rendimientos logarítmicos de la simulación
simulated_log_returns = np.diff(np.log(simulated_path))

Veamos ahora el comportamiento de los datos simulados.

In [ ]:
simulator.plot_simulation()

### Comparación Gráfica de las Distribuciones

Visualizaremos ambas distribuciones (real y simulada) mediante histogramas y estimaciones de densidad kernel (KDE). Un buen ajuste se reflejará en una gran superposición entre ambas curvas.

In [ ]:
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1) # 1 row, 2 columns, 1st plot
plt.hist(log_returns, bins=100, alpha=0.8, color='royalblue')
plt.title('S&P 500 Log Returns (Real)')
plt.xlabel('Log Returns')
plt.ylabel('Frequency')
plt.grid(True)

plt.subplot(1, 2, 2) # 1 row, 2 columns, 2nd plot
plt.hist(simulated_log_returns, bins=100, alpha=0.8, color='orangered')
plt.title('Simulated Log Returns')
plt.xlabel('Log Returns')
plt.ylabel('Frequency')
plt.grid(True)

plt.tight_layout() # Adjust layout to prevent overlapping
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))

# Histograma y KDE para los datos reales del S&P 500
sns.histplot(log_returns, bins=100, kde=True, stat='density', color='royalblue', label='S&P 500 (Real)')

# Histograma y KDE para los datos simulados
sns.histplot(simulated_log_returns, bins=100, kde=True, stat='density', color='orangered', alpha=0.7, label='Simulado (con parámetros estimados)')

plt.title('Comparación de la Distribución de Rendimientos: Real vs. Simulado', fontsize=16)
plt.xlabel('Rendimientos Logarítmicos', fontsize=12)
plt.ylabel('Densidad', fontsize=12)
plt.legend()
plt.show()

### Conclusión
(En construcción, aún no se cumple)

El gráfico de comparación muestra que la distribución de los rendimientos simulados se asemeja bastante a la de los datos reales. En particular, el modelo parece capturar bien:

-   El **pico central** alrededor de cero, que representa los días de baja volatilidad.
-   Las **colas más pesadas** (leptocurtosis) que un modelo normal, indicando que los eventos extremos son más probables de lo que una distribución gaussiana sugeriría.

Esto valida que el modelo de difusión con saltos es una herramienta significativamente mejor que un modelo de difusión simple para capturar la dinámica de los mercados financieros.